In [1]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

In [2]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
import math
import builtins
from google.cloud import storage
from google.cloud import *

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_gcp import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ModuleNotFoundError: No module named 'inverted_index_gcp'

In [3]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
# from graphframes import *

In [ ]:
pandasDf = pd.read_csv("/content/newdata.csv")
pandasDf = pandasDf.drop(columns = "Unnamed: 0")
df_data = spark.createDataFrame(pandasDf)

pages_links = df_data.select('id', 'anchor_text').rdd



In [4]:
# @app.route("/get_pagerank", methods=['POST'])

from io import BytesIO
    
def get_pagerank():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
#     wiki_ids = request.get_json()
#     if len(wiki_ids) == 0:
#         return jsonify(res)
    bucket_name = '208987248' 
    wiki_ids = [3434750]
    client = storage.Client()
    file_name = "pr_part-00000-c1f636d8-972a-483c-a516-d1b60f8e868f-c000.csv"
    bucket = client.get_bucket(bucket_name)
    blob = bucket.get_blob(f'pr/' + file_name)
    content = blob.download_as_string()
    df = pd.read_csv(BytesIO(content), header = None)
    df.columns = ["id","pageRank"]
#     print(df)
    data = spark.createDataFrame(df)
    for i in wiki_ids:
        try:
            res.append(df[i])
            print(data[i])
        except:
            continue
#     print(res)
    return res
#     return jsonify(res)

In [5]:
print(get_pagerank())

[]


In [ ]:
# @app.route("/get_pageview", methods=['POST'])

bucket_name = '208987248' 

def open_gcp(file_name):
    client = storage.Client(file_name)
    bucket = client.bucket(bucket_name)
    blob = bucket.get_blob(f'pv/' + file_name)
    return blob.open('rb')


def read_pickle(file_name):
    stream = open_gcp(file_name+".pkl")
    pick = pickle.load(stream)
    stream.close()
    return pick

def get_pageview():
    ''' Returns PageRank values for a list of provided wiki article IDs. 

        Test this by issuing a POST request to a URL like:
          http://YOUR_SERVER_DOMAIN/get_pagerank
        with a json payload of the list of article ids. In python do:
          import requests
          requests.post('http://YOUR_SERVER_DOMAIN/get_pagerank', json=[1,5,8])
        As before YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io
        if you're using ngrok on Colab or your external IP on GCP.
    Returns:
    --------
        list of floats:
          list of PageRank scores that correrspond to the provided article IDs.
    '''
    res = []
    wiki_ids = request.get_json()
    if len(wiki_ids) == 0:
        return jsonify(res)
    wid2pv = read_pickle('pageviews-202108-user')
    for i in wiki_ids:
        res.append(wid2pv[i])
    return jsonify(res)

In [ ]:
# print(get_pageview())

# Run the app

In [ ]:
# you need to upload your implementation of search_app.py
import search_frontend as se

In [ ]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
# import importlib
# importlib.reload(se)

In [ ]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://XXXX-XX-XX-XX-XX.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))